**Installing Transformer Library**

In [5]:
!pip install transformers torch jinja2

**USING GOOGLE DRIVE TO MOUNT SAVED PROMPT FILES HERE IN COLAB**

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/student_career_recommender.zip /content/
!unzip -o student_career_recommender.zip
%cd student_career_recommender


Archive:  student_career_recommender.zip
   creating: student_career_recommender/
  inflating: student_career_recommender/.DS_Store  
  inflating: __MACOSX/student_career_recommender/._.DS_Store  
  inflating: student_career_recommender/requirements.txt  
  inflating: __MACOSX/student_career_recommender/._requirements.txt  
  inflating: student_career_recommender/career_mapping.json  
  inflating: __MACOSX/student_career_recommender/._career_mapping.json  
  inflating: student_career_recommender/README.md  
  inflating: __MACOSX/student_career_recommender/._README.md  
   creating: student_career_recommender/prompts/
  inflating: student_career_recommender/prompts/fallback_prompt.txt  
  inflating: __MACOSX/student_career_recommender/prompts/._fallback_prompt.txt  
  inflating: student_career_recommender/prompts/recommendation_prompt.txt  
  inflating: __MACOSX/student_career_recommender/prompts/._recommendation_prompt.txt  
  inflating: student_career_recommender/prompts/extract_prefe

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from jinja2 import Template
import json
import torch


**Load the Model**

In [8]:
model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


**LOAD AND RENDER PROMPT TEMPLATE**

In [9]:
def load_prompt(path, variables: dict):
    with open(path, 'r') as f:
        template = Template(f.read())
    return template.render(**variables)


**STUDENT INPIUT**

In [10]:
student_input = """
Hi! I love painting and sketching in my free time. I enjoy biology and scored well in it last year.
I'm not much into sports but I enjoy watching documentaries and designing digital art.
"""


**EXTRACT PREFRENCES FROM PROMPT**

In [11]:
pref_prompt = load_prompt("prompts/extract_preferences.txt", {"student_input": student_input})

response = pipe(pref_prompt, max_new_tokens=250)[0]["generated_text"]
print("🔍 Extracted Info:\n", response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🔍 Extracted Info:
 You are a friendly academic counselor. Based on the following conversation, extract the student's key interests, hobbies, and academic strengths.

Conversation:

Hi! I love painting and sketching in my free time. I enjoy biology and scored well in it last year. 
I'm not much into sports but I enjoy watching documentaries and designing digital art.


Return a summary like:
- Interests:
- Hobbies:
- Academic strengths:
- Not interested in sports:
- Enjoys watching documentaries and designing digital art:
```python
Summary = {}

# extract key information from conversation
student_interests = "painting, sketching, biology"
student_hobbies = "watching documentaries, designing digital art"
student_academic_strengths = "scored well in biology"
student_not_interested = "sports"
student_other_interests = "watching documentaries and designing digital art"

# store in summary variable
Summary['Interests'] = student_interests
Summary['Hobbies'] = student_hobbies
Summary['Academi

**FALLBACK PROMPT**

In [12]:
if "Interests:" not in response:
    fallback = open("prompts/fallback_prompt.txt").read()
    clarification = pipe(fallback, max_new_tokens=100)[0]["generated_text"]
    print("⚠️ Fallback needed:\n", clarification)


**Manually Simulate Parsed Output- For Testing**

In [13]:
parsed = {
    "interests": "biology, documentaries",
    "hobbies": "painting, sketching, digital art",
    "strengths": "biology"
}


**Load Career Mapping and Match Field**

In [15]:
with open("career_mapping.json", "r") as f:
    mapping = json.load(f)

def get_paths(parsed, mapping):
    fields = []
    for item in parsed["interests"].split(",") + parsed["hobbies"].split(",") + parsed["strengths"].split(","):
        item = item.strip().lower()
        if item in mapping:
            fields.extend(mapping[item])
    return list(set(fields))

career_paths = get_paths(parsed, mapping)
print(" Suggested Career Paths:\n", career_paths)


 Suggested Career Paths:
 ['Medicine']


**Generate Career Recommendation Prompt**

In [16]:
rec_prompt = load_prompt("prompts/recommendation_prompt.txt", {
    "interests": parsed["interests"],
    "hobbies": parsed["hobbies"],
    "strengths": parsed["strengths"]
})

rec_response = pipe(rec_prompt, max_new_tokens=300)[0]["generated_text"]
print("📢 Career Recommendation:\n", rec_response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


📢 Career Recommendation:
 You are a career advisor. Given these preferences:

- Interests: biology, documentaries
- Hobbies: painting, sketching, digital art
- Academic strengths: biology

Recommend 2–3 suitable career paths from: STEM, Arts, Sports, Commerce, Design, Law, Medicine.

Also provide a short explanation (2-3 lines) for why each is suitable.

<|question_end|>Career Exam Solutions

Answer 1:
Step 1: Identify your interests and talents. In this case, the individual is interested in biology and has a talent for sketching and digital art.
Step 2: Research different occupations that combine biology and art. For example, a career as a Wildlife Illustrator would allow the individual to use their artistic talent to create illustrations of various species, while also having a deep interest in biology.
Step 3: Explore education and training requirements for the chosen careers. For example, pursuing a degree in Fine Arts with a focus on illustration would be a good path for someone in

**Saving The Output For References**

In [18]:
with open("final_recommendation.txt", "w") as f:
    f.write(rec_response)
